In [5]:
from simple_lama_inpainting import SimpleLama
from PIL import Image, ImageChops, ImageDraw
import numpy as np
import cv2

# Function to narrow the mask (erosion)
def narrow_mask(mask, iterations=1):
    mask_array = np.array(mask)
    kernel = np.ones((3, 3), np.uint8)
    narrowed_mask = mask_array
    for _ in range(iterations):
        narrowed_mask = cv2.erode(narrowed_mask, kernel, iterations=1)
    return Image.fromarray(narrowed_mask)

# Function to expand the mask (dilation)
def expand_mask(mask, iterations=1):
    mask_array = np.array(mask)
    kernel = np.ones((3, 3), np.uint8)
    expanded_mask = mask_array
    for _ in range(iterations):
        expanded_mask = cv2.dilate(expanded_mask, kernel, iterations=1)
    return Image.fromarray(expanded_mask)

# Initialize SimpleLama
simple_lama = SimpleLama()

# Load image and initial mask
img_path = "images/image.png"
mask_path = "images/mask.png"

image = Image.open(img_path)
mask = Image.open(mask_path).convert('L')

ITERATIONS = 5

# Step 1: Narrow the mask (shrink the inpainting region)
mask = narrow_mask(mask, iterations=ITERATIONS)  # Narrow the mask

# Step 2: Progressive inpainting by expanding the mask
for step in range(ITERATIONS +1):  # Number of progressive steps
    result = simple_lama(image, mask)
    mask = expand_mask(mask, iterations=1)  # Expand mask gradually
    image = result  # Use the result as input for the next step

# Save the final result
result.save(f"images/inpainted_progressive_{ITERATIONS}.png")
